In [55]:
# Codice per estrarre i supporti dei materiali con meno di 100 occorrenze in totale.
# Vengono estratte le occorrenze sia per uomini che per donne, in modo da poter mantenere
# il rapporto iniziale.

import os

path_dataset_men = "C:\\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\men"
path_dataset_women = "C:\\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\women"

imgs_list_men = [os.path.join(path_dataset_men, img_name) for img_name in sorted(os.listdir(path_dataset_men))]
imgs_list_women = [os.path.join(path_dataset_women, img_name) for img_name in sorted(os.listdir(path_dataset_women))]


to_augment_men = {'polyurethane': 0, 'canvas': 0, 'polyester': 0, 'cow-leather': 0, 'raffia': 0, 'goat-leather': 0}
to_augment_women = {'polyurethane': 0, 'canvas': 0, 'polyester': 0, 'cow-leather': 0, 'raffia': 0, 'goat-leather': 0}

for image_path in imgs_list_men:
	material = image_path.split('\\').pop().split("_").pop().split(".")[0]
	if material in to_augment_men.keys():
		to_augment_men[material] += 1


print("men = " + str(to_augment_men))

for image_path in imgs_list_women:
	material = image_path.split('\\').pop().split("_").pop().split(".")[0]
	if material in to_augment_women.keys():
		to_augment_women[material] += 1

print("women = " + str(to_augment_women))

men = {'polyurethane': 15, 'canvas': 3, 'polyester': 13, 'cow-leather': 3, 'raffia': 1, 'goat-leather': 0}
women = {'polyurethane': 19, 'canvas': 15, 'polyester': 11, 'cow-leather': 84, 'raffia': 19, 'goat-leather': 13}


In [56]:
to_augment_men = {'polyurethane': 15, 'canvas': 3, 'polyester': 13, 'cow-leather': 3, 'raffia': 1, 'goat-leather': 0}
to_augment_women = {'polyurethane': 19, 'canvas': 15, 'polyester': 11, 'cow-leather': 84, 'raffia': 19, 'goat-leather': 13}
to_augment_total = {'polyurethane': 34, 'canvas': 18, 'polyester': 24, 'cow-leather': 87, 'raffia': 20, 'goat-leather': 13}

In [57]:
def calculate_number_of_image_to_create(to_augment_men, to_augment_women, to_augment_total):
	total_image_to_create = 100
	result = {}
	for material in to_augment_total.keys():
		number_of_image_to_create = total_image_to_create - to_augment_total[material]
		ratio_woman = to_augment_women[material]/to_augment_total[material]
		ratio_man = to_augment_men[material]/to_augment_total[material]
		number_of_image_to_create_woman = round(number_of_image_to_create * ratio_woman)
		number_of_image_to_create_man = round(number_of_image_to_create * ratio_man)
		result[material+"_women"] = number_of_image_to_create_woman
		result[material+"_men"] = number_of_image_to_create_man
	return result



In [58]:
print(calculate_number_of_image_to_create(to_augment_men, to_augment_women, to_augment_total))

{'polyurethane_women': 37, 'polyurethane_men': 29, 'canvas_women': 68, 'canvas_men': 14, 'polyester_women': 35, 'polyester_men': 41, 'cow-leather_women': 13, 'cow-leather_men': 0, 'raffia_women': 76, 'raffia_men': 4, 'goat-leather_women': 87, 'goat-leather_men': 0}


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array, load_img

def augment_data(img, path_to_save, brand_name, materials, number_of_supports_to_create, n_image):
	datagen = ImageDataGenerator(
			rotation_range = 60,
			zoom_range = 0.1,
			brightness_range = (0.5,1.5),
			horizontal_flip = True,
			vertical_flip = True,
			shear_range = 0.2
			)

	img = load_img(img)
# Converting the input sample image to an array
	x = img_to_array(img)
	# Reshaping the input image
	x = x.reshape((1, ) + x.shape)

	# Generating and saving 5 augmented samples
	# using the above defined parameters.
	index = 0
	while number_of_supports_to_create > 0 :
		for batch in datagen.flow(x, batch_size = 1,
								save_to_dir = path_to_save,
								save_prefix = str(n_image+ index)+"_"+brand_name+"_"+materials, save_format ='jpg'):
			break
		number_of_supports_to_create -= 1
		index += 1


In [59]:
path_augmented_men = "C:\\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\\augmented_men"
path_augmented_women = "C:\\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\\augmented_women"

imgs_list = imgs_list_men + imgs_list_women


to_create = calculate_number_of_image_to_create(to_augment_men, to_augment_women,to_augment_total)

id_contatore = 1402

for image in imgs_list:
	img_material = image.split('\\').pop().split("_").pop().split(".")[0]
	if img_material in to_augment_total.keys():
		img_brand = image.split('\\').pop().split("_")[1]

		if '\\'.join(image.split("\\")[:-1]) == path_dataset_men:
			number_of_this_image_to_create = round(to_create[img_material+"_men"]/to_augment_men[img_material])
			print(img_material + " da creare: " +str(number_of_this_image_to_create))
			augment_data(image, path_augmented_men, img_brand, img_material, number_of_this_image_to_create, id_contatore)
			id_contatore += number_of_this_image_to_create


		elif '\\'.join(image.split("\\")[:-1]) == path_dataset_women:
			number_of_this_image_to_create = round(to_create[img_material+"_women"]/to_augment_women[img_material])
			augment_data(image, path_augmented_women, img_brand, img_material, number_of_this_image_to_create, id_contatore)
			id_contatore += number_of_this_image_to_create

polyurethane da creare: 2
polyester da creare: 3
polyurethane da creare: 2
polyurethane da creare: 2
polyester da creare: 3
polyurethane da creare: 2
polyurethane da creare: 2
polyester da creare: 3
polyurethane da creare: 2
polyurethane da creare: 2
polyester da creare: 3
polyester da creare: 3
polyurethane da creare: 2
polyester da creare: 3
raffia da creare: 4
canvas da creare: 5
polyurethane da creare: 2
canvas da creare: 5
polyester da creare: 3
polyester da creare: 3
canvas da creare: 5
polyurethane da creare: 2
polyester da creare: 3
polyurethane da creare: 2
polyester da creare: 3
polyurethane da creare: 2
polyester da creare: 3
polyester da creare: 3
cow-leather da creare: 0
polyurethane da creare: 2
polyurethane da creare: 2
polyester da creare: 3
cow-leather da creare: 0
cow-leather da creare: 0
polyurethane da creare: 2


In [60]:
import hashlib
import os
def remove_duplicates(dataset_path):
	hashes = set()
	equals = []
	for filename in sorted(os.listdir(dataset_path)):
		path = os.path.join(dataset_path, filename)
		digest = hashlib.sha1(open(path,'rb').read()).digest()
		if digest not in hashes:
			hashes.add(digest)
		else:
			print("Trovato un duplicato.")
			os.remove(path)
			print("Rimosso duplicato: "+ path)
			equals.append(path)
	print("I duplicati in " +dataset_path +" erano: ")
	print(equals)

In [61]:
remove_duplicates('C:\\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\\augmented_men')
remove_duplicates('C:\\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\\augmented_women')

I duplicati in C:\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\augmented_men erano: 
[]
I duplicati in C:\Users\gioel\PycharmProjects\CVandDL\dataset_classificazione\augmented_women erano: 
[]


In [62]:
# Rename definitivo (datagen.flow creava immagini col prefisso giusto ma con un suffisso in più

for filename in sorted(os.listdir(path_augmented_women)):
	os.rename(os.path.join(path_augmented_women,filename), os.path.join(path_augmented_women,'_'.join(filename.split("_")[:-2])+".jpg"))

for filename in sorted(os.listdir(path_augmented_men)):
	os.rename(os.path.join(path_augmented_men,filename), os.path.join(path_augmented_men,'_'.join(filename.split("_")[:-2])+".jpg"))